Deliverables:

1-You must create a simple web API to predict a user's BMI in real-time. You can also use webcam input. The goal is to use one of the pre-trained image models (e.g. VGG Face), fine-tune with the provided data, and deploy via jupyter notebook, streamlit, flask or any other simple restful api's.

2-10 pages of the write-up about your implementation.

3-10 mins presentation or live demo in the final lecture.

Our goal is to beat the performance metrics provided in the paper.

In [38]:
import os
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive/')
path_gdrive = '/content/drive/MyDrive/Colab Datasets/ML/BMI'
os.chdir(path_gdrive)
print(os.getcwd())

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Datasets/ML/BMI


In [39]:
#!unzip "/content/drive/MyDrive/Colab Datasets/ML/BMI-20230313T174553Z-001.zip"

In [40]:
import sys
import os
import json
import pandas as pd
import numpy as np
import optparse

from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, SimpleRNN, Masking, Embedding
from keras.utils import pad_sequences
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict

In [41]:
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

In [42]:
# read csv file
data_path = '/content/drive/MyDrive/Colab Datasets/ML/BMI/Data/data.csv'
bmi = pd.read_csv(data_path, index_col=0)

In [43]:
# check csv
bmi.head()

,bmi,gender,is_training,name
0,34.207396,Male,1,img_0.bmp
1,26.453720,Male,1,img_1.bmp
2,34.967561,Female,1,img_2.bmp
3,22.044766,Female,1,img_3.bmp
4,37.758789,Female,1,img_4.bmp


In [59]:
bmi.is_training.value_counts()

1    3368
0     838
Name: is_training, dtype: int64

In [58]:
bmi['imgae_type'] = bmi['name'].str.extract(r'\.(\w+)$')
bmi['imgae_type'].value_counts()

bmp    4206
Name: imgae_type, dtype: int64

In [17]:
# seperate training data
#train_df = bmi[bmi.is_training == 1]
#validation_df = bmi[bmi.is_training == 0]

In [18]:
#train_df.head()

,bmi,gender,is_training,name
0,34.207396,Male,1,img_0.bmp
1,26.453720,Male,1,img_1.bmp
2,34.967561,Female,1,img_2.bmp
3,22.044766,Female,1,img_3.bmp
4,37.758789,Female,1,img_4.bmp


In [37]:
#train_df.to_csv('/content/drive/MyDrive/Colab Datasets/ML/BMI/Data/train.csv', index=False)
#validation_df.to_csv('/content/drive/MyDrive/Colab Datasets/ML/BMI/Data/val.csv', index=False)

In [28]:
!pip show keras

Name: keras
Version: 2.12.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: keras-vggface, tensorflow


In [34]:
!python --version

Python 3.10.11


In [35]:
# https://stackoverflow.com/questions/68862735/keras-vggface-no-module-named-keras-engine-topology
! pip install git+https://github.com/rcmalli/keras-vggface.git
! pip install keras_applications --no-deps

filename = "/usr/local/lib/python3.10/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))

import tensorflow as tf
from keras_vggface.vggface import VGGFace

vggface = VGGFace(model='resnet50') # or VGGFace() as default

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-96b602um
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-96b602um
  Resolved https://github.com/rcmalli/keras-vggface.git to commit bee35376e76e35d00aeec503f2f242611a97b38a
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
165439116/165439116 [==============================] - 1s 0us/step


In [51]:
!ls

Data


In [67]:
# code reference to https://github.com/abhaymise/Face-to-height-weight-BMI-estimation-/blob/master/BMI%20prediction.ipynb

profile_df = bmi.copy()

# map image to obvervation in bmi table
data_folder = "Data/Images"

from glob import glob
all_files = glob(data_folder+"/*")
all_imgs = sorted([img for img in all_files if ".bmp" in img])
print("Total {} photos ".format(len(all_jpgs)))

Total 3963 photos 


In [77]:
def get_index_of_digit(string):
    import re
    digits = re.findall('\d+', string)
    return digits[0]

# def get_index_of_digit(string):
#     import re
#     match = re.search("\d", p(string).stem)
#     return match.start(0)

test = 'Data/Images/img_1882.bmp'
get_index_of_digit(test)

'1882'

In [74]:
p('1882').stem

'1882'

In [80]:
from pathlib import Path as p
id_path = [(get_index_of_digit(images),images) for  images in all_imgs]
image_df = pd.DataFrame(id_path,columns=['id','path'])


In [81]:
# reset index
image_df = image_df.set_index('id')

,id,path
0,0,Data/Images/img_0.bmp
1,1,Data/Images/img_1.bmp
2,10,Data/Images/img_10.bmp
3,100,Data/Images/img_100.bmp
4,1000,Data/Images/img_1000.bmp
...,...,...
3958,995,Data/Images/img_995.bmp
3959,996,Data/Images/img_996.bmp
3960,997,Data/Images/img_997.bmp
3961,998,Data/Images/img_998.bmp


In [57]:
len(id_path)

3963

Note:

1. Transfer Learning. https://medium.com/@leosimmons/estimating-body-mass-index-from-face-images-using-keras-and-transfer-learning-de25e1bc0212